In [1]:
import numpy as np
import subprocess

# Root estimation 

In this notebook we describe the pipeline we use to do the experiments from Section 4.2. 

1) Simulate data sets
2) Do inference using mcmc algorithm 
3) Assess convergence 
4) Compute mean squared error and compare to mean and phylogenetic mean 

##### 0. Define settings for experiment

In [ ]:
rootpath = 'data/hercules_forewing_n=20.csv'
treepath = 'data/chazot_subtree.nw'
outpath = '_root-experiment'
sigma_sim = 0.9
alpha_sim = 0.02
gamma = 0.001
n_datasets = 5
dt = 0.05
sti=1

tau_sigma = 0.2
tau_alpha = 0.005
prior_sigma = (0.7, 1.3)
prior_alpha = (0.0005, 0.03)
n_iter = 100 #this is 3000 in the paper
lambd = 0.8
ls = 2 # super root branch length 
x_s = 'phylomean'

##### 1. Simulate data sets

In [3]:
for i in range(n_datasets):
  subprocess.run(['python', 'simulate_data.py', 
                '-dt', f'{dt}',
                '-a', f'{alpha_sim}',
                '-s', f'{sigma_sim}',
                '-ov', f'{gamma}',
                '-root', f'{rootpath}',
                '-o', f'{outpath}/simdata', 
                '-simtree', f'{treepath}', 
                '-sti', f'{sti}', # whether or not do the stratonovich ito corr
                '-rb', '0'  
                  ])

!! no data seed given
simulation seed: 49873795413804327


here() starts at /Users/lkn315/Documents/stoch_phyl_mod_shape


> args <- commandArgs(trailingOnly = TRUE)
> 
> # do what we want 
> tree = read.tree(paste(args[1], '.nw', sep=''))
> vcv_ = vcv(tree)
> write.table(vcv_, file=paste(args[1],'_vcv.csv', sep=''), row.names=F, col.names=F)
> 
!! no data seed given
simulation seed: 7680677238277473


here() starts at /Users/lkn315/Documents/stoch_phyl_mod_shape


> args <- commandArgs(trailingOnly = TRUE)
> 
> # do what we want 
> tree = read.tree(paste(args[1], '.nw', sep=''))
> vcv_ = vcv(tree)
> write.table(vcv_, file=paste(args[1],'_vcv.csv', sep=''), row.names=F, col.names=F)
> 
!! no data seed given
simulation seed: 12238695649958997


...

> args <- commandArgs(trailingOnly = TRUE)
> 
> # do what we want 
> tree = read.tree(paste(args[1], '.nw', sep=''))
> vcv_ = vcv(tree)
> write.table(vcv_, file=paste(args[1],'_vcv.csv', sep=''), row.names=F, col.names=F)
> 
!! no data seed given
simulation seed: 59484663473074378


here() starts at /Users/lkn315/Documents/stoch_phyl_mod_shape


> args <- commandArgs(trailingOnly = TRUE)
> 
> # do what we want 
> tree = read.tree(paste(args[1], '.nw', sep=''))
> vcv_ = vcv(tree)
> write.table(vcv_, file=paste(args[1],'_vcv.csv', sep=''), row.names=F, col.names=F)
> 
!! no data seed given
simulation seed: 10953749833646560


here() starts at /Users/lkn315/Documents/stoch_phyl_mod_shape


> args <- commandArgs(trailingOnly = TRUE)
> 
> # do what we want 
> tree = read.tree(paste(args[1], '.nw', sep=''))
> vcv_ = vcv(tree)
> write.table(vcv_, file=paste(args[1],'_vcv.csv', sep=''), row.names=F, col.names=F)
> 


##### 2. Run MCMC to infer posterior 

We show in the notebook how to run MCMC but in practice we do not run MCMC from a notebook and this is just to give an example of how to run the mcmc.py script.  

A few comments regarding the mcmc.py script: 

Some of the variables in the code are not named in accordance with the paper (see below):
- gtheta = sigma 
- kalpha = alpha 
- obs_var = gamma 
- mirrored Gaussian = reflected Gaussian

In the paper we update parameters by evaluating $g_s(x_s; \theta)$ in the code we refer to this as "logrhorilde" and not g_s. 

In [4]:
with open (f'{outpath}.sh', 'w') as rsh:
    rsh.write(f'''#!/bin/bash
read seed
              
screen -md -S {outpath} python mcmc.py -N {n_iter} -l {lambd} -dt {dt} -datapath {outpath}/simdata -tau_sigma {tau_sigma} -tau_alpha {tau_alpha} -palpha {prior_alpha[0]} {round(prior_alpha[1]-prior_alpha[0],2)} -psigma {prior_sigma[0]} {round(prior_sigma[1]-prior_sigma[0],2)} -ov {gamma} -super_root {x_s} -o {outpath}/runs -ds $seed
screen -md -S {outpath} python mcmc.py -N {n_iter} -l {lambd} -dt {dt} -datapath {outpath}/simdata -tau_sigma {tau_sigma} -tau_alpha {tau_alpha} -palpha {prior_alpha[0]} {round(prior_alpha[1]-prior_alpha[0],2)} -psigma {prior_sigma[0]} {round(prior_sigma[1]-prior_sigma[0],2)} -ov {gamma} -super_root {x_s} -o {outpath}/runs -ds $seed
screen -md -S {outpath} python mcmc.py -N {n_iter} -l {lambd} -dt {dt} -datapath {outpath}/simdata -tau_sigma {tau_sigma} -tau_alpha {tau_alpha} -palpha {prior_alpha[0]} {round(prior_alpha[1]-prior_alpha[0],2)} -psigma {prior_sigma[0]} {round(prior_sigma[1]-prior_sigma[0],2)} -ov {gamma} -super_root {x_s} -o {outpath}/runs -ds $seed

'''
    )

We runs the bash script generated above from the commandline. 

##### 3. Assess convergence and plot convergence diagnostics

We first convergence by running the diagnostics.py plot. After that all convergence diagnostics in one plot using the notebook 2-root-experiment/diagnostics_all_exp.ipynb. 

In [ ]:
#! ls _root-experiment/runs | while read seed; do python diagnostics.py -folder_runs _root-experiment/runs/$seed -folder_simdata _root-experiment/simdata/$seed -MCMC_iter 100 -burnin 10 -nnodes 9 -simtree data/chazot_subtree.nw; done 

We plot a summary of the convergence diagnostics by running the script in the folder "2-root-experiment" named "diagnostis_all_exp.ipynb". 

##### 5. Compute mean squared error

We compute the mean squared error between mean, phylogenetic mean, posterior mean, posterior mode and posterior median by running the notebook 4.2-root-experiment/roots-estimation.ipynb.